In [1]:
# paquetes para importar
import requests
import json
import numpy as np
import datetime
import string
import pandas as pd

## a) Primer intento de modelo

In [2]:
df_resume = pd.read_csv('my_data.csv')

In [3]:
# https://medium.com/mlearning-ai/forecast-with-xgboost-a1abf870c1cc
# seguimos ese ejemplo con los datos tal y como los tenemos

In [4]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
import chart_studio
import chart_studio.plotly as py 

In [5]:
df_resume['date'] = pd.DatetimeIndex(df_resume['date']).date
df_resume['year'] = pd.DatetimeIndex(df_resume['date']).year
df_resume['month'] = pd.DatetimeIndex(df_resume['date']).month
df_resume.head()

,date,hour,value,year,month
0,2014-04-01,0,92.75,2014,4
1,2014-04-01,1,86.46,2014,4
2,2014-04-01,2,78.82,2014,4
3,2014-04-01,3,77.47,2014,4
4,2014-04-01,4,76.90,2014,4


In [6]:
X, y =  df_resume.loc[:,['month', 'year']].values, df_resume.loc[:,'value'].values
data_dmatrix = xgb.DMatrix(X,label=y)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [8]:
reg_mod = xgb.XGBRegressor(objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.10,
    subsample=0.5,
    colsample_bytree=1, 
    max_depth=5,
)
reg_mod.fit(X_train, y_train)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor=None, random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [9]:
reg_mod.fit(X_train,y_train)

predictions = reg_mod.predict(X_test)

In [10]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("RMSE: %f" % (rmse))

from sklearn.metrics import r2_score
r2 = np.sqrt(r2_score(y_test, predictions))
print("R_Squared Score : %f" % (r2))

from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test, predictions)
print("MAPE : %f" % (mape))

RMSE: 27.064327
R_Squared Score : 0.883698
MAPE : 0.103809


In [11]:
# obtenemos un modelo con un R2 del 81.94% y un RMSE del 33.14%
## ---> parece satisfactorio 

## habría que mejorar añadiendo días festivos y día de la semana para ver cómo mejora

# Creating an empty Dataframe with column names only
result = pd.DataFrame(columns=['Time', 'Test', 'Predicted'])
result['Time'] = pd.date_range(start='1/4/2014' , periods=14175)
result['Test'] = y_test
result['Predicted'] = predictions
#Using Plotly to build the graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=result['Time'], y=result['Test'],
                    mode='lines',
                    name='Test'))
fig.add_trace(go.Scatter(x=result['Time'], y=result['Predicted'],
                    mode='lines',
                    name='Predicted'))

# Edit the layout
fig.update_layout(title='Test vs Predicted Values',
                   xaxis_title='Months',
                   yaxis_title='Electricity (GWh)',
                   template='gridon')

fig.show()

df=pd.DataFrame(predictions, columns=['pred']) 
df['date'] = pd.date_range(start='2/5/2022', periods=len(df), freq='M')
fig = px.line(df, x="date", y="pred")
# Edit the layout
fig.update_layout(title='China Electricity Consumption Forecast',
                   xaxis_title='Date',
                   yaxis_title='Electricity - GWh',
                   template='gridon')
fig.show()

### b) Con nuevo dataset

In [12]:
df_resume = pd.read_csv('my_data_version_1.csv')

In [13]:
df_resume['date'] = pd.DatetimeIndex(df_resume['date']).date
df_resume['year'] = pd.DatetimeIndex(df_resume['date']).year
df_resume['month'] = pd.DatetimeIndex(df_resume['date']).month
df_resume.head()

,date,hour,value,weekday,working_day,year,month
0,2014-04-01,0,92.75,1,1,2014,4
1,2014-04-01,1,86.46,1,1,2014,4
2,2014-04-01,2,78.82,1,1,2014,4
3,2014-04-01,3,77.47,1,1,2014,4
4,2014-04-01,4,76.90,1,1,2014,4


In [14]:
X, y =  df_resume.loc[:,['month', 'year']].values, df_resume.loc[:,'value'].values
data_dmatrix = xgb.DMatrix(X,label=y)

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [16]:
reg_mod = xgb.XGBRegressor(objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.10,
    subsample=0.5,
    colsample_bytree=1, 
    max_depth=5,
)
reg_mod.fit(X_train, y_train)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor=None, random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
reg_mod.fit(X_train,y_train)

predictions = reg_mod.predict(X_test)

In [18]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("RMSE: %f" % (rmse))

from sklearn.metrics import r2_score
r2 = np.sqrt(r2_score(y_test, predictions))
print("R_Squared Score : %f" % (r2))

from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test, predictions)
print("MAPE : %f" % (mape))

RMSE: 27.064327
R_Squared Score : 0.883698
MAPE : 0.103809


In [19]:
# hemos mejorado bastante la precisión del modelo añadiendo los días de vacaciones y días de la semana

### c) Dataset con temperaturas

In [20]:
df_resume = pd.read_csv('my_data_version_2.csv')

In [21]:
df_resume['date'] = pd.DatetimeIndex(df_resume['date']).date
df_resume['year'] = pd.DatetimeIndex(df_resume['date']).year
df_resume['month'] = pd.DatetimeIndex(df_resume['date']).month
df_resume.head()

,date,hour,value,weekday,working_day,madrid_temp,barcelona_temp,bilbao_temp,sevilla_temp,valencia_temp,year,month
0,2014-04-01,0,92.75,1,1,6.93180,10.72770,12.90347,13.18472,11.11930,2014,4
1,2014-04-01,1,86.46,1,1,6.22500,10.48672,12.94668,13.17422,10.26528,2014,4
2,2014-04-01,2,78.82,1,1,6.36930,10.27407,12.83950,13.30044,9.36490,2014,4
3,2014-04-01,3,77.47,1,1,6.54214,9.92593,12.53115,13.32388,8.63247,2014,4
4,2014-04-01,4,76.90,1,1,6.66910,9.79630,12.17030,13.25796,8.16445,2014,4


In [22]:
X, y =  df_resume.loc[:,['month', 'year']].values, df_resume.loc[:,'value'].values
data_dmatrix = xgb.DMatrix(X,label=y)

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [24]:
reg_mod = xgb.XGBRegressor(objective='reg:squarederror',
    n_estimators=1000,
    learning_rate=0.10,
    subsample=0.5,
    colsample_bytree=1, 
    max_depth=5,
)
reg_mod.fit(X_train, y_train)
XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor=None, random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.5,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [25]:
reg_mod.fit(X_train,y_train)

predictions = reg_mod.predict(X_test)

In [26]:
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print("RMSE: %f" % (rmse))

from sklearn.metrics import r2_score
r2 = np.sqrt(r2_score(y_test, predictions))
print("R_Squared Score : %f" % (r2))

from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test, predictions)
print("MAPE : %f" % (mape))

RMSE: 23.525137
R_Squared Score : 0.867910
MAPE : 0.098258
